# 2.5 复杂指令-检测风力发电机

## 复杂任务执行方式
- 人工观测，无人机一步一步执行。基本使用
- 人工制定完整工作流程，无人机一步一步执行。workflow
- 无人机自行观测，自行执行。Agent，具身智能。

## 本节任务-风电叶片检测

具体需求：
- 无人机需要能够在飞手配合下完成任务。完成叶片检测。
- 接受一次指令，执行一次，然后等待下一次指令
- 指令相对简单，没有复杂的任务

In [9]:
#构建新的提示词，知识库写入aisim_lession25.txt
kg_promot_file = "prompts/aisim_lession25.txt"

kg_prompt = """
以下是您可以用来命令无人机的一些功能。

aw.takeoff() - 起飞无人机。
aw.land() - 无人机着陆。
aw.get_drone_position() - 以与 X、Y、Z 坐标相对应的 3 个浮点数的列表形式返回无人机的当前位置。
aw.fly_to([x, y, z]) - 将无人机飞到指定的位置，该位置由对应于 X、Y、Z 坐标的三个参数列表指定。
aw.get_position(object_name)：采用字符串作为输入，指示感兴趣对象的名称，并返回 3 个浮点数的列表，指示其 X、Y、Z 坐标。

场景中存在以下对象，您将使用这些确切的名称来引用它们：

风力发电机1、风力发电机2、太阳能电池板、汽车、人群、塔1、塔2、塔3。

请注意，在函数调用的时候，请使用对应的英文名称，具体对应如下：
turbine1 ：风力发电机1
turbine2 ：风力发电机2
solarpanels：太阳能电池板
car：汽车
crowd:人群
tower1:塔1
tower2:塔2
tower3:塔3

例如，你需要获得风力发电机1的位置，你可以这样写：
aw.get_position("turbine1")
而不是：
aw.get_position("风力发电机1")

就坐标轴约定而言，我们使用的NED，也就是"北东地"坐标系，
，即+X是北，+Y是东，+Z是下。这意味着Z值越高越负，如果原点在地面上，z值是零，地面以上是负的！向上飞是z轴减去一个值

除了无人机本身之外，所有物体都不可移动。请记住，有两个涡轮机和三个塔。当有多个相同类型的对象时，
如果我没有明确指定我指的是哪个对象，您应该始终要求我进行澄清。永远不要做出假设。

回复均按照如下例子的格式： 
```python
i=1#这里输出python代码
```
此代码功能为赋值

你无需考虑aw的import问题，在环境中已经声明
"""

pt_file = open(kg_promot_file, "w", encoding="utf-8")
pt_file.write(kg_prompt)
pt_file.close()

In [1]:
import airsim_agent
my_agent = airsim_agent.AirSimAgent(knowledge_prompt="prompts/aisim_lession25.txt")

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [2]:
command = "起飞"
python_code = my_agent.process(command, True) #执行代码
print("python_code: \n", python_code)

python_code: 
 aw.takeoff()



In [3]:
command = "请飞向风力发电机1，沿X轴方向，在风力发电机10米之前停下来"
python_code = my_agent.process(command, True) #执行代码
print("python_code: \n", python_code)

python_code: 
 # 获取风力发电机1的位置
turbine1_position = aw.get_position("turbine1")
# 计算目标位置，沿X轴方向在风力发电机1位置前10米
target_x = turbine1_position[0] - 10
target_y = turbine1_position[1]
target_z = turbine1_position[2]
target_position = [target_x, target_y, target_z]
# 让无人机飞到目标位置
aw.fly_to(target_position)



In [4]:
command = "继续向风力发电机2飞行。沿 X 轴保持距离风力发电机2 12米，飞向风力发电机2，高度达到 50 米。"
python_code = my_agent.process(command, True) #执行代码
print("python_code: \n", python_code)

python_code: 
 turbine2_pos = aw.get_position("turbine2")
target_x = turbine2_pos[0] - 12
target_y = turbine2_pos[1]
target_z = -50  # 高度50米，因为+Z是下，所以取负
aw.fly_to([target_x, target_y, target_z])#此代码功能为让无人机沿X轴保持距离风力发电机2 12米飞向它，且高度达到50米



In [5]:
command = "现在让我们开始检查风力发电机的叶片。第一个叶片垂直向上，请飞到当前位置上方 30 米，然后再次向下,注意向上飞Z轴是减。"
python_code = my_agent.process(command, True) #执行代码
print("python_code:", python_code)

python_code: current_pos = aw.get_drone_position()
target_up_z = current_pos[2] - 30
target_up_pos = [current_pos[0], current_pos[1], target_up_z]
aw.fly_to(target_up_pos)
aw.fly_to(current_pos)#此代码功能为让无人机飞到当前位置上方30米，然后再回到当前位置



In [6]:
command = "为了检查第二个叶片，我需要你以与水平轴Y成30度角的右下方向飞行。您应该在YZ 平面上飞行30 米的距离，然后返回当前位置。"
python_code = my_agent.process(command, True) #执行代码
print("python_code:", python_code)

python_code: import math

# 获取当前无人机位置
current_pos = aw.get_drone_position()

# 计算目标位置
distance = 30
angle = math.radians(30)  # 将角度转换为弧度

# 在YZ平面上计算偏移量
delta_y = distance * math.cos(angle)
delta_z = distance * math.sin(angle)

# 计算目标位置
target_y = current_pos[1] + delta_y
target_z = current_pos[2] + delta_z
target_x = current_pos[0]
target_pos = [target_x, target_y, target_z]

# 飞向目标位置
aw.fly_to(target_pos)

# 返回当前位置
aw.fly_to(current_pos)
# 此代码功能为让无人机以与水平轴Y成30度角的右下方向在YZ平面飞行30米，然后返回当前位置



In [7]:
command = "为了检查第三个叶片，我需要你以与水平轴Y成30度角的左下方向飞行。您应该在YZ 平面上飞行30 米的距离，然后返回当前位置。"
python_code = my_agent.process(command, True) #执行代码
print("python_code:", python_code)

python_code: import math

# 获取无人机当前位置
current_pos = aw.get_drone_position()

# 计算在 YZ 平面上飞行 30 米，与水平轴 Y 成 30 度角（左下方向）的偏移量
distance = 30
angle = math.radians(30)
delta_y = distance * math.cos(angle)
delta_z = distance * math.sin(angle)

# 计算目标位置，左下方向 Y 轴是减少，Z 轴是增加
target_y = current_pos[1] - delta_y
target_z = current_pos[2] + delta_z
target_x = current_pos[0]
target_pos = [target_x, target_y, target_z]

# 飞向目标位置
aw.fly_to(target_pos)

# 返回当前位置
aw.fly_to(current_pos)
# 此代码功能为让无人机以与水平轴 Y 成 30 度角的左下方向在 YZ 平面飞行 30 米，然后返回当前位置



In [9]:
command = "现在我们把检查叶片的三个动作串联起来。我需要你重复刚才所做的事情，但到了最后就停下来，无需返回。"
python_code = my_agent.process(command, True) #执行代码
print("python_code:", python_code)

python_code: import math

# 起飞
aw.takeoff()

# 飞向风力发电机1沿X轴方向10米前的位置
turbine1_pos = aw.get_position("turbine1")
target_x = turbine1_pos[0] - 10
target_y = turbine1_pos[1]
target_z = turbine1_pos[2]
aw.fly_to([target_x, target_y, target_z])

# 飞向风力发电机2，沿X轴保持距离12米，高度50米
turbine2_pos = aw.get_position("turbine2")
target_x = turbine2_pos[0] - 12
target_y = turbine2_pos[1]
target_z = -50
aw.fly_to([target_x, target_y, target_z])

# 检查第一个叶片，飞到当前位置上方30米
current_pos = aw.get_drone_position()
target_up_z = current_pos[2] - 30
target_up_pos = [current_pos[0], current_pos[1], target_up_z]
aw.fly_to(target_up_pos)
aw.fly_to(current_pos)

# 检查第二个叶片，以与水平轴Y成30度角的右下方向飞行30米
distance = 30
angle = math.radians(30)
delta_y = distance * math.cos(angle)
delta_z = distance * math.sin(angle)
target_y = current_pos[1] + delta_y
target_z = current_pos[2] + delta_z
target_x = current_pos[0]
target_pos = [target_x, target_y, target_z]
aw.fly_to(target_pos)
aw.fly_to(current_pos)

# 检查第三个叶片，以与水平轴Y成30度角的左下方向飞行30米，最后

In [10]:
command = "现在向前移动，飞到风力发电机后面。在 X 轴上飞行 20 米，再加上几米的间隙，然后将无人机旋转 180 度。您可以使用 set_yaw() 函数来旋转无人机。"
python_code = my_agent.process(command, True) #执行代码
print("python_code:", python_code)

python_code: # 获取当前无人机位置
current_pos = aw.get_drone_position()

# 假设间隙为 5 米，计算目标位置
target_x = current_pos[0] + 20 + 5
target_y = current_pos[1]
target_z = current_pos[2]
target_pos = [target_x, target_y, target_z]

# 飞到目标位置
aw.fly_to(target_pos)

# 将无人机旋转 180 度
aw.set_yaw(180)
# 此代码功能为让无人机飞到风力发电机后面，在 X 轴上飞行 20 米并加上 5 米间隙，然后将无人机旋转 180 度



In [11]:
command = "太好了！现在我要你检查叶片的背面。这意味着你所做的与之前所做的完全相反，也就是说，首先朝右上角的方向飞行30米，然后停下。"
python_code = my_agent.process(command, True) #执行代码
print("python_code:", python_code)

python_code: import math

# 获取当前无人机位置
current_pos = aw.get_drone_position()

# 计算在 YZ 平面上飞行 30 米，与水平轴 Y 成 30 度角（右上角方向）的偏移量
distance = 30
angle = math.radians(30)
delta_y = distance * math.cos(angle)
delta_z = -distance * math.sin(angle)  # 右上角方向 Z 轴是减少

# 计算目标位置
target_y = current_pos[1] + delta_y
target_z = current_pos[2] + delta_z
target_x = current_pos[0]
target_pos = [target_x, target_y, target_z]

# 飞向目标位置
aw.fly_to(target_pos)
# 此代码功能为让无人机朝右上角方向在 YZ 平面飞行 30 米后停下，用于检查叶片背面



In [12]:
command = "接下来巡检第一个叶片，其垂直向上，请飞到当前位置上方 30 米，然后再次向下,注意向上飞Z轴是减。"
python_code = my_agent.process(command, True) #执行代码
print("python_code:", python_code)

python_code: # 获取当前无人机位置
current_pos = aw.get_drone_position()

# 计算向上 30 米后的目标位置
target_up_z = current_pos[2] - 30
target_up_pos = [current_pos[0], current_pos[1], target_up_z]

# 飞到上方 30 米处
aw.fly_to(target_up_pos)

# 飞回当前位置
aw.fly_to(current_pos)
# 此代码功能为让无人机飞到当前位置上方 30 米，然后再回到当前位置，用于巡检垂直向上的第一个叶片



In [14]:
command = "为了检查第二个叶片，我需要你以与水平轴Y成30度角的右下方向飞行。您应该在YZ 平面上飞行30 米的距离，然后返回当前位置。"
python_code = my_agent.process(command, True) #执行代码
print("python_code:", python_code)

python_code: import math

# 获取无人机当前位置
current_pos = aw.get_drone_position()

# 计算偏移量
distance = 30
angle = math.radians(30)
delta_y = distance * math.cos(angle)
delta_z = distance * math.sin(angle)

# 计算目标位置，右下方向 Y 轴增加，Z 轴增加
target_y = current_pos[1] + delta_y
target_z = current_pos[2] + delta_z
target_x = current_pos[0]
target_pos = [target_x, target_y, target_z]

# 飞向目标位置
aw.fly_to(target_pos)

# 返回当前位置
aw.fly_to(current_pos)
# 此代码功能为让无人机以与水平轴 Y 成 30 度角的右下方向在 YZ 平面飞行 30 米，然后返回当前位置，用于检查第二个叶片

